In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score

In [3]:
train = pd.read_csv('../input/wine-preferences/train_cleaned.csv', converters={"text": lambda x: ' '.join(x.strip("[]").replace("'", '').split(", "))}, index_col=0)
test = pd.read_csv('../input/wine-preferences/train_cleaned.csv', converters={"text":  lambda x: ' '.join(x.strip("[]").replace("'", '').split(", "))}, index_col=0)

In [4]:
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

In [5]:
train.head()

,label,text
0,1,kid start prek read learn read thought great k...
1,0,purchas ear amazon last week less go amazon pr...
2,1,ive use skeeter stik number year highli allerg...
3,1,ive vacuum littl two year ive consist impress ...
4,0,heard rage pilat zumba fan kickbox fan thought...


Sentiment labels of the Amazon dataset - resume

In [7]:
train.groupby('label').count()

,text
label,
0,1800000
1,1800000


In [10]:
test.groupby('label').count()

,text
label,
0,1800000
1,1800000


In [ ]:
cv = CountVectorizer(binary = True)
cv.fit(train['text'])

In [ ]:
#import pickle
#with open('vmodel.txt', 'wb') as m:
#    pickle.dump(cv, m)

In [ ]:
train_x = cv.transform(train['text'])
test_x = cv.transform(test['text'])

In [ ]:
xgb_train = xgb.DMatrix(train_x, train['label'])


## Model with custom parameters

In [ ]:
param = {'eta': 0.75,
         'max_depth': 50,
         'objective': 'binary:logitraw'}

xgb_model = xgb.train(param, xgb_train, num_boost_round = 30)

In [ ]:
#xgb_test = xgb.DMatrix(test_x, test['label'])
xgb_test = xgb.DMatrix(test_x)

y_pred = xgb_model.predict(xgb_test)
y_pred = np.where(np.array(y_pred) > 0.5, 1, 0)

In [ ]:
print(accuracy_score(test['label'], y_pred))    
print(f1_score(test['label'], y_pred))           

In [ ]:
xgb_model.save_model('big_model.json')